In [4]:
import json
import pandas as pd
import os

#set working directory to github folder, change if on mac / other computer
os.chdir("/Users/skoebric/Dropbox/GitHub/gtgfoliummap")

allcountriesjson = "/Users/skoebric/Dropbox/GitHub/gtgfoliummap/geometry/allcountries.geojson"

#opens the json with outlines of all countries, creates 'dump' which contains a list of all countries geography/properties
openfile = open(allcountriesjson, 'r')
json_input = json.load(openfile)
dump = json_input['features']

#%%
#pulls csv from static google sheet location, passes it to df
CSVURL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRkObK4sgJUYpz4XfHS3KG8teVYY6aWCO-efLM6Qrk0GkP_MNmsx5MLTcSo0QDRhlorEr1GSvp7Mc2F/pub?gid=0&single=true&output=csv"
import requests
import io

data = requests.get(CSVURL)
df = pd.read_csv(io.StringIO(data.text))

#%%
#checks the df and creates two lists, one for 'pilot projects', one for 'other assistance'
#and one dict 'whereweworkdictlist' with a list of properties for each 
whereweworkindexed = df.set_index('Country')
whereweworkindexed['Name'] = whereweworkindexed.index
#whereweworkdictlist = wherewework.to_dict(orient = 'records')

pilotprojects = list(whereweworkindexed[whereweworkindexed['Assistance Type'] == 'Technical assistance partnership']['Name'])
#otherbens = list(whereweworkindexed[whereweworkindexed['Assistance Type'] == 'Other Assistance']['Name'])
#%%     
#create 'whereweworkout' with the geography and properties for each country in lists
whereweworkout = []
for x in dump:
#    if (x['properties']['name']) in otherbens:
#        y = whereweworkindexed.loc[x['properties']['name']].dropna()
#        x['properties'] = y.to_dict()
#        whereweworkout.append(x)
    if (x['properties']['name']) in pilotprojects:
        y = whereweworkindexed.loc[x['properties']['name']].dropna()
        x['properties'] = y.to_dict()
        whereweworkout.append(x)

#%%
#export geojson
outdict = {"type": "FeatureCollection",
                     "features":whereweworkout}


#save the pilotprojectsout list to the 'wherewework.geojson' file
outfile = "/Users/skoebric/Dropbox/GitHub/gtgfoliummap/geometry/wherewework.geojson"
with open(outfile, 'w') as outpath:
    json.dump(outdict, outpath)

KeyError: 'Assistance Type'

In [6]:
import folium
import subprocess
import webbrowser
import sys


json = "/Users/skoebric/Dropbox/GitHub/gtgfoliummap/geometry/wherewework.geojson"

m = folium.Map(width = '100%', height = 800, location = (20,5),zoom_start = 3,
               no_wrap=True,max_bounds=True, min_zoom=3, tiles="OpenStreetMap")
tt = folium.GeoJsonTooltip(['Name','Toolkit'], aliases = ['Country','Toolkits'])
style_function = lambda x: {'fillColor': '#73ad02',
                            'color': '#73ad02',
                            'weight':1,
                            }

folium.GeoJson(json, tooltip = tt, name = 'json', style_function = style_function, highlight_function = style_function).add_to(m)
m.save("/Users/skoebric/Dropbox/GitHub/gtgfoliummap/index.html")
m